In [1]:
import numpy as np
import pandas as pd
import glob
import os
import matplotlib.pyplot as plt
from scipy import signal
import sys

In [2]:
def GetRMSE(x2, y2, x1, y1):
    from scipy.spatial.distance import pdist

    er = []
    for idx in range(len(x2)):
        X = np.asarray([[x1[idx], y1[idx]], 
                         [x2[idx], y2[idx]]])
        temp_er = pdist(X,metric = 'euclidean')
        er.append(temp_er[0])
    er = np.asarray(er)
    return(er)

In [3]:
# parameters used to filter data

cutoff = 24

interpol_order = 3 # order for polynomial interpolation

# win_gauss = signal.gaussian(10,3) # gaussian kernal for smoothening interpolated data

# parameters for the savitzky-golay filter
savgol_win = 15
savgol_polyorder = 3

## remove x y location of points based on distance from previous frame threshold 

In [4]:
direc = r"../dataFolders/PaperPipelineOutput/RawTracks/"
for visitnum in ['FirstVisit/', 'LastVisit/']:
    
    # get all the data for that visit num
    path = os.path.join(direc, visitnum)
    trackslist = glob.glob(path + '*.csv')
    
    new_path = os.path.join(r"../dataFolders/PaperPipelineOutput/FilteredTracks/", visitnum)
    try:
        os.mkdir(new_path)
    except OSError:
        print('oops')
    
    # loop through all the moths in the list
    for data in trackslist:
        name = os.path.basename(data)[:-4]
        file = pd.read_csv(data)
        x = file.x.values
        y = file.y.values
        
        x_notinView = x <=5
        y_notinView = y <=5

        x[x_notinView & y_notinView]=np.nan
        y[x_notinView & y_notinView]=np.nan

        # get rmse values for subsequent frames
        rmse = GetRMSE(x[1:], y[1:], x[:-1], y[:-1])

        filtered_x = np.copy(x[1:])
        filtered_y = np.copy(y[1:])

        filtered_x[rmse > cutoff] = np.nan
        filtered_y[rmse > cutoff] = np.nan

        new_xypts = pd.DataFrame({'filtered_x': filtered_x, 'filtered_y': filtered_y, 'rmse': rmse})
        new_xypts.to_csv(new_path + '/' + name + '.csv')

oops


C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:33: RuntimeWarning: invalid value encountered in greater
C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:34: RuntimeWarning: invalid value encountered in greater


oops


## get angle and magnitude

In [5]:
def Unitvector(x_gauss, y_gauss):
    from sklearn import preprocessing
    
    # get the slope of the tangent
    trajectory = np.asarray([x_gauss, y_gauss])
    m = np.gradient(trajectory, axis = 1)
    m_atx = m[1]/m[0]
    
    # get the tangent vector at x = x0 + 1
    tangent_x = x_gauss+1
    tangent_y = m_atx + y_gauss
    
    # get the unit tangent vector
    u_x = []
    u_y = []

    for x,y,x0,y0 in zip(tangent_x, tangent_y, x_gauss, y_gauss):
        if np.any(np.isnan([x, y])) or np.any(np.isinf([x, y])):
            unit_x = np.nan
            unit_y = np.nan
        else:
            vector = np.asarray([x-x0, y-y0]).reshape(1,-1)
            [unit_x, unit_y] = preprocessing.normalize(vector, norm = 'l2')[0]
        u_x.append(unit_x)
        u_y.append(unit_y)
    u_x = np.asarray(u_x)
    u_y = np.asarray(u_y)
    return(u_x, u_y)

def getAngle(loc, tangent):
    cross = np.cross(tangent, loc)
    dot = np.dot(tangent, loc)
    angle = np.arctan2(cross, dot)*180/np.pi
    return(angle)

def wrapAngle(angle):
    angle = np.absolute(angle)
    for i,a in enumerate(angle):
        if a > 90:
            a = 180 - a
            angle[i] = a
    return(angle)

In [6]:
circ_parameters_path = glob.glob('../dataFolders/PaperPipelineOutput/CircleParameters/' + '*.csv')
circ_parameters = pd.read_csv(circ_parameters_path[0])

In [7]:
visit = 'FirstVisit/'
direc = os.path.join('../dataFolders/PaperPipelineOutput/FilteredTracks/', visit)
datalist = glob.glob(direc + '*.csv')

In [8]:
newpath = os.path.join('../dataFolders/PaperPipelineOutput/RadiusAndAngle/', visit)
try:
    os.mkdir(newpath)
except OSError:
    print('oops')

oops


In [9]:
for path in datalist:
        
        file = pd.read_csv(path)
        x = file.filtered_x
        y = file.filtered_y

        name = [n for n in circ_parameters.name if n in path][0]
        
        if x.size == 0 or y.size == 0:
            print(name + 'has emtpy x y tracks')
            continue

        circ_x = circ_parameters.loc[circ_parameters.name == name, 'circ_x'].values
        circ_y = circ_parameters.loc[circ_parameters.name == name, 'circ_y'].values
        circ_radii = circ_parameters.loc[circ_parameters.name == name, 'circ_radii'].values

        cent_x = x - circ_x
        cent_y = y - circ_y

        x_interpl = cent_x.interpolate(method='polynomial', order=interpol_order)
        y_interpl = cent_y.interpolate(method='polynomial', order=interpol_order)
        
        x_interpl = x_interpl[~np.isnan(x_interpl)]
        y_interpl= y_interpl[~np.isnan(y_interpl)]

        r = np.linalg.norm([x_interpl, y_interpl], axis = 0)
        r = r/circ_radii
        
        # savitzky-golay method
        x_savgol = signal.savgol_filter(x_interpl, savgol_win, savgol_polyorder)
        y_savgol = signal.savgol_filter(y_interpl, savgol_win, savgol_polyorder)
        r_savgol = np.linalg.norm([x_savgol, y_savgol], axis = 0)
        r_savgol_norm = r_savgol/circ_radii

        # save all usable variables as series

        df1 = pd.Series(data = x_savgol, name = 'x_savgol')
        df2 = pd.Series(data = y_savgol, name = 'y_savgol')
        df3 = pd.Series(data = r_savgol_norm, name = 'radial distance savgol')

        
        #calculate the unit tangent vectors - savitzky-golay vector
        u_x, u_y = Unitvector(x_savgol, y_savgol)

        angle_savgol = []
        for x0, y0, x, y in zip(x_savgol, y_savgol, u_x, u_y):
            loc = [x0, y0]
            tangent = [x, y]
            a = getAngle(loc, tangent)
            angle_savgol.append(a)
            
        angle_savgol = wrapAngle(angle_savgol)

        df4 = pd.Series(data = angle_savgol, name = 'angle_savgol')
       
        new_file = pd.concat([file, df1, df2, df3, df4], axis = 1)
        new_file.to_csv(newpath + name + 'RadiusAndAngle.csv', index_label = False)

C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: divide by zero encountered in true_divide
  import sys
C:\Users\Tanvi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:7: RuntimeWarning: invalid value encountered in true_divide
  import sys


## check where are we getting NaN's in the interpolated data

In [10]:
for path in datalist:
        
        file = pd.read_csv(path)
        x = file.filtered_x
        y = file.filtered_y

        name = [n for n in circ_parameters.name if n in path][0]        
        
        circ_x = circ_parameters.loc[circ_parameters.name == name, 'circ_x'].values
        circ_y = circ_parameters.loc[circ_parameters.name == name, 'circ_y'].values
        circ_radii = circ_parameters.loc[circ_parameters.name == name, 'circ_radii'].values

        cent_x = x - circ_x
        cent_y = y - circ_y

        x_interpl = cent_x.interpolate(method='polynomial', order=interpol_order)
        y_interpl = cent_y.interpolate(method='polynomial', order=interpol_order)
        
        print(name + str(np.where(np.isnan(x_interpl))))
        print(name + str(np.where(np.isnan(y_interpl))))

c-10_m10(array([315, 316, 317, 318, 319], dtype=int64),)
c-10_m10(array([315, 316, 317, 318, 319], dtype=int64),)
c-10_m11(array([   0, 1733], dtype=int64),)
c-10_m11(array([   0, 1733], dtype=int64),)
c-10_m12(array([], dtype=int64),)
c-10_m12(array([], dtype=int64),)
c-10_m13(array([183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193], dtype=int64),)
c-10_m13(array([183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193], dtype=int64),)
c-10_m15(array([259], dtype=int64),)
c-10_m15(array([259], dtype=int64),)
c-10_m16_cropped(array([], dtype=int64),)
c-10_m16_cropped(array([], dtype=int64),)
c-10_m17_cropped(array([], dtype=int64),)
c-10_m17_cropped(array([], dtype=int64),)
c-10_m18_cropped(array([   0, 3855], dtype=int64),)
c-10_m18_cropped(array([   0, 3855], dtype=int64),)
c-10_m19_cropped(array([958], dtype=int64),)
c-10_m19_cropped(array([958], dtype=int64),)
c-10_m1(array([ 18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  28,  29,  30,
        31,  32,  33,  34,  35,  36,  37

c-2_m12(array([], dtype=int64),)
c-2_m12(array([], dtype=int64),)
c-2_m13_cropped(array([], dtype=int64),)
c-2_m13_cropped(array([], dtype=int64),)
c-2_m14_cropped(array([0, 1, 2, 3, 4, 5], dtype=int64),)
c-2_m14_cropped(array([0, 1, 2, 3, 4, 5], dtype=int64),)
c-2_m15(array([], dtype=int64),)
c-2_m15(array([], dtype=int64),)
c-2_m16_cropped(array([], dtype=int64),)
c-2_m16_cropped(array([], dtype=int64),)
c-2_m17(array([], dtype=int64),)
c-2_m17(array([], dtype=int64),)
c-2_m18(array([], dtype=int64),)
c-2_m18(array([], dtype=int64),)
c-2_m1(array([], dtype=int64),)
c-2_m1(array([], dtype=int64),)
c-2_m20(array([], dtype=int64),)
c-2_m20(array([], dtype=int64),)
c-2_m21_cropped(array([], dtype=int64),)
c-2_m21_cropped(array([], dtype=int64),)
c-2_m22(array([], dtype=int64),)
c-2_m22(array([], dtype=int64),)
c-2_m23(array([], dtype=int64),)
c-2_m23(array([], dtype=int64),)
c-2_m2(array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18